# Generate Images 3D
Jan 22, 2021

In [1]:
import argparse
import os
import sys

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data

from torchsummary import summary
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

from torch.utils.data import DataLoader, TensorDataset

import time
from datetime import datetime
import glob
import pickle
import yaml
import collections

In [2]:
%matplotlib widget

In [3]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/4_basic_3d_GAN/1_main_code/')
from utils import *

## Build Model structure

In [4]:
def f_load_config(config_file):
    with open(config_file) as f:
        config = yaml.load(f, Loader=yaml.SafeLoader)
    return config

def f_init_gdict(gdict,config_dict):
    ''' Initialize the global dictionary gdict with values in config file'''
    keys1=['workers','nc','nz','ngf','ndf','beta1','kernel_size','stride','g_padding','d_padding','flip_prob']
    keys2=['image_size','checkpoint_size','num_imgs','ip_fname','op_loc']
    for key in keys1: gdict[key]=config_dict['training'][key]
    for key in keys2: gdict[key]=config_dict['data'][key]

In [5]:
def f_gen_images(gdict,netG,optimizerG,ip_fname,op_loc,op_strg='inf_img_',op_size=500):
    '''Generate images for best saved models
     Arguments: gdict, netG, optimizerG, 
                 ip_fname: name of input file
                op_strg: [string name for output file]
                op_size: Number of images to generate
    '''

    nz,device=gdict['nz'],gdict['device']

    try:
        if torch.cuda.is_available(): checkpoint=torch.load(ip_fname)
        else: checkpoint=torch.load(ip_fname,map_location=torch.device('cpu'))
    except Exception as e:
        print(e)
        print("skipping generation of images for ",ip_fname)
        return
    
    ## Load checkpoint
    if gdict['multi-gpu']:
        netG.module.load_state_dict(checkpoint['G_state'])
    else:
        netG.load_state_dict(checkpoint['G_state'])
    
    ## Load other stuff
    iters=checkpoint['iters']
    epoch=checkpoint['epoch']
    optimizerG.load_state_dict(checkpoint['optimizerG_state_dict'])
    
    # Generate batch of latent vectors
    noise = torch.randn(op_size, 1, 1, 1, nz, device=device)
    # Generate fake image batch with G
    netG.eval() ## This is required before running inference
    gen = netG(noise)
    gen_images=gen.detach().cpu().numpy()
    print(gen_images.shape)
    
    op_fname='%s_epoch-%s_step-%s.npy'%(op_strg,epoch,iters)

    np.save(op_loc+op_fname,gen_images)

    print("Image saved in ",op_fname)

In [6]:
if __name__=="__main__":
    torch.backends.cudnn.benchmark=True
    t0=time.time()
    #################################
#     args=f_parse_args()
    # Manually add args ( different for jupyter notebook)
    args=argparse.Namespace()
    args.config='1_main_code/config_3d.yaml'
    args.ngpu=1
    args.batchsize=128
#     args.spec_loss_flag=True
#     args.checkpoint_size=50
    args.epochs=1
    args.learn_rate=0.0002
#     args.mode='fresh'
#     args.run_suffix='_nb_test'
#     args.deterministic=False
#     args.seed='36723705'
#     args.lambda1=5.0
#     args.save_steps_list=[5,10]

    ### Set up ###
    config_file=args.config
    config_dict=f_load_config(config_file)

    # Initilize variables    
    gdict={}
    f_init_gdict(gdict,config_dict)
    
    ## Add args variables to gdict
    for key in ['ngpu','batchsize','epochs','learn_rate']:
        gdict[key]=vars(args)[key]
    
    gdict['device']=torch.device("cuda" if (torch.cuda.is_available() and gdict['ngpu'] > 0) else "cpu")
    gdict['ngpu']=torch.cuda.device_count()
    gdict['multi-gpu']=True if (gdict['device'].type == 'cuda') and (gdict['ngpu'] > 1) else False 

    print("Building GAN networks")
    # Create Generator
    netG = Generator(gdict).to(gdict['device'])
    netG.apply(weights_init)
    #     print(netG)
    # summary(netG,(1,1,64))
    
    print("Number of GPUs used %s"%(gdict['ngpu']))
    if (gdict['multi-gpu']):
        netG = nn.DataParallel(netG, list(range(gdict['ngpu'])))

    optimizerG = optim.Adam(netG.parameters(), lr=gdict['learn_rate'], betas=(gdict['beta1'], 0.999),eps=1e-7)


Building GAN networks
Number of GPUs used 0


## Run Inference

In [9]:
ls /global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/

20210223_210217_3dcgan_predict_0.8_m2/  20210227_050213_3dcgan_predict_0.65_m2/


In [8]:
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/'
# fldr='20210227_050213_3dcgan_predict_0.65_m2'
# op_loc=main_dir+fldr+'/images/'
# ip_fname=main_dir+fldr+'/models/checkpoint_22010.tar'
# f_gen_images(gdict,netG,optimizerG,ip_fname,op_loc,op_strg='inference_spec',op_size=500)

[Errno 2] No such file or directory: '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/20210121_133144_3d_l0.5/models/checkpoint_22010.tar'
skipping generation of images for  /global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/20210121_133144_3d_l0.5/models/checkpoint_22010.tar


In [10]:
## For multiple steps 
main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/'
fldr='20210227_050213_3dcgan_predict_0.65_m2'

step_list=[27600, 30950, 42620, 43830, 50440, 58320, 58330, 59500, 64670,65600]
for step in step_list:
    for count in range(2): # Repeat inference
        try: 
            op_loc=main_dir+fldr+'/images/'
            ip_fname=main_dir+fldr+'/models/checkpoint_{0}.tar'.format(step)
            f_gen_images(gdict,netG,optimizerG,ip_fname,op_loc,op_strg='inference_%s'%(count),op_size=1000)
        except Exception as e:
            print(e)
            print("skipping ",step)


[Errno 2] No such file or directory: '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/20210227_050213_3dcgan_predict_0.65_m2/models/checkpoint_27600.tar'
skipping generation of images for  /global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/20210227_050213_3dcgan_predict_0.65_m2/models/checkpoint_27600.tar
[Errno 2] No such file or directory: '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/20210227_050213_3dcgan_predict_0.65_m2/models/checkpoint_27600.tar'
skipping generation of images for  /global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/20210227_050213_3dcgan_predict_0.65_m2/models/checkpoint_27600.tar
Error(s) in loading state_dict for Generator:
	size mismatch for main.0.weight: copying a param with shape torch.Size([32768, 65]) from checkpoint, the shape in current model is torch.Size([32768, 64]).
skipping 

In [ ]:
# fname=op_loc+'inference_spec_epoch-11_step-37040.npy'
# a1=np.load(fname)
# print(a1.shape)